In [12]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Date, ForeignKey

In [13]:
from sqlalchemy import URL
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import create_engine, text

# Create a MetaData instance
metadata = MetaData()

url_object = URL.create(
    "mysql+pymysql",
    username="root",
    password="Ranjeeta1@",  # plain (unescaped) text
    host="localhost",
    database="sales_data_json_new",
)


# Create a metadata object
metadata = MetaData()

engine = create_engine(url_object)

In [14]:
# Now connect to create the database if it doesn't exist
with engine.connect() as conn:
    # Create the database if it doesn't exist
    conn.execute(text("CREATE DATABASE IF NOT EXISTS sales_data_json_new;"))
    
    # Optionally, connect to the newly created database
    conn.execute(text("USE sales_data_json_new;"))
    
    # Show all databases to confirm creation
    result = conn.execute(text("SHOW DATABASES;"))
    print(result.fetchall())

# Now you can create tables in the new database as needed

[('information_schema',), ('mysql',), ('performance_schema',), ('sales_data',), ('sales_data_json',), ('sales_data_json_new',), ('sys',)]


In [15]:
# from sqlalchemy import URL
# from sqlalchemy import create_engine, MetaData, text

# # Create a URL object without specifying the database
# url_object = URL.create(
#     "mysql+pymysql",
#     username="root",
#     password="Ranjeeta1@",  # Use plain text
#     host="localhost",
# )

# # Create a metadata object
# metadata = MetaData()

# # Create the engine without the database
# engine = create_engine(url_object)

# # Now connect to create the database if it doesn't exist
# with engine.connect() as conn:
#     # Create the database if it doesn't exist
#     conn.execute(text("CREATE DATABASE IF NOT EXISTS sales_data_json_new;"))
    
#     # Optionally, connect to the newly created database
#     conn.execute(text("USE sales_data_json_new;"))
    
#     # Show all databases to confirm creation
#     result = conn.execute(text("SHOW DATABASES;"))
#     print(result.fetchall())

# # Now you can create tables in the new database as needed


In [16]:
# Bind metadata to the engine
metadata = MetaData()

# Define tables
product = Table('Product', metadata,
    Column('product_id', String(100), primary_key=True),
    Column('category', String(100)),
    Column('sub_category_encoded', String(100))
)

client = Table('Client', metadata,
    Column('client_id', String(100), primary_key=True),
    Column('birth', Date),
    Column('sex', String(1)),
)

sale = Table('Sale', metadata,
    Column('sale_id', Integer, primary_key=True, autoincrement=True),  # Adding an auto-incrementing ID
    Column('product_id', String(100), ForeignKey('Product.product_id')),
    Column('client_id', String(100), ForeignKey('Client.client_id')),
    Column('quantity_sold', Integer),
    Column('sale_date', Date)
)

# Create tables in the database by explicitly passing the engine
metadata.create_all(engine)

In [17]:
# Ensure the connection is correct
with engine.connect() as conn:
    result = conn.execute(text("SELECT DATABASE();"))
    print(f"Connected to database: {result.fetchone()[0]}")  # Should be 'sales_data_json'

Connected to database: sales_data_json_new


### to check for all existing tables

In [18]:
from sqlalchemy import inspect

inspector = inspect(engine)

# Get list of tables
tables = inspector.get_table_names()
print("Tables in database:", tables)

# Get details of a specific table
for table in tables:
    print(f"\nStructure of table '{table}':")
    columns = inspector.get_columns(table)
    for column in columns:
        print(f"{column['name']} - {column['type']}")

Tables in database: ['client', 'product', 'sale']

Structure of table 'client':
client_id - VARCHAR(100)
birth - DATE
sex - VARCHAR(1)

Structure of table 'product':
product_id - VARCHAR(100)
category - VARCHAR(100)
sub_category_encoded - VARCHAR(100)

Structure of table 'sale':
sale_id - INTEGER
product_id - VARCHAR(100)
client_id - VARCHAR(100)
quantity_sold - INTEGER
sale_date - DATE


In [19]:
clients_df = pd.read_json('data-json/clients.json')
produits_df = pd.read_json('data-json/produits_sous-categorie.json')
ventes_df = pd.read_json('data-json/ventes.json')

In [21]:
# Insert data into tables
with engine.connect() as connection:
    # Insert products
    for _, row in produits_df.iterrows():
        ins = product.insert().prefix_with("IGNORE").values(
            product_id=row['product_id'],
            category=row['category'],
            sub_category_encoded=row['sub_category']
        )
        connection.execute(ins)

    # Insert clients
    for _, row in clients_df.iterrows():
        ins = client.insert().prefix_with("IGNORE").values(
            client_id=row['client_id'],
            birth=pd.to_datetime(row['birth']).date(),
            sex=row['sex'],
            # sex_numeric=row['sex_numeric']
        )
        connection.execute(ins)

    # Insert sales
    for _, row in sales_df.iterrows():
        ins = sale.insert().values(
            sale_id=row['sale_id'],
            product_id=row['product_id'],
            client_id=row['client_id'],
            quantity_sold=row['quantity_sold'],
            sale_date=pd.to_datetime(row['date']).date()
        )
        connection.execute(ins)

    print("Data insertion completed successfully!")

    # Verify data insertiona
    print("\nSample Products:")
    result = connection.execute(product.select().limit(5))
    for row in result:
        print(row)

    print("\nSample Clients:")
    result = connection.execute(client.select().limit(5))
    for row in result:
        print(row)

    print("\nSample Sales:")
    result = connection.execute(sale.select().limit(5))
    for row in result:
        print(row)